In [1]:
import sys, os, natsort, glob

import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch, datasets
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F
from PIL import Image

import mediapipe as mp
import utils
import LantentNet
cudnn.enabled = True

gpu = 0 # GPU ID

#Initialize Model
model = LantentNet.LantentNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 66)
saved_state_dict = torch.load('models\Latent_model_0,999.pkl') # 0,999 model for best yaw results
model.load_state_dict(saved_state_dict)



def load_filtered_state_dict(model, snapshot):
    # By user apaszke from discuss.pytorch.org
    model_dict = model.state_dict()
    snapshot = {k: v for k, v in snapshot.items() if k in model_dict}
    model_dict.update(snapshot)
    model.load_state_dict(model_dict)


#Classification and Regression losses
criterion = nn.CrossEntropyLoss().cuda(gpu)
reg_criterion = nn.MSELoss().cuda(gpu)


MAE = torch.nn.L1Loss(reduction='mean')


softmax = nn.Softmax().cuda(gpu)

#For classification vector
idx_tensor = [idx for idx in range(66)]
idx_tensor = Variable(torch.FloatTensor(idx_tensor)).cuda(gpu)


model.cuda(gpu)
model.eval() #Inference mode

print("done")

done


# AFLW2000 OCCLUDED DATASET

In [21]:
image_path = natsort.natsorted(glob.glob('datasets\Data_Ocluded_AFLW2000_Full_Faces\*.jpg'))

pose_path = natsort.natsorted(glob.glob('datasets\AFLW2000-3D\AFLW2000\*.mat'))

transformations =  transforms.Compose([transforms.Resize(260), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

test_dataset = datasets.new_Pose_AFLW2000(image_path, pose_path, transformations)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=1,shuffle=False, num_workers=0)
print(len(image_path))
print('done')

# TEST OCCLUDED AFLW2000

In [22]:
MAE_LATENT_y = np.array([])
MAE_LATENT_p = np.array([])
MAE_LATENT_r = np.array([])


num_epochs = 1
with torch.no_grad():
    for i, (img, labels, cont_labels) in enumerate(test_loader):

        img = Variable(img).cuda(gpu)

        label_yaw_cont = Variable(cont_labels[:,0]).cuda(gpu)
        label_pitch_cont = Variable(cont_labels[:,1]).cuda(gpu)
        label_roll_cont = Variable(cont_labels[:,2]).cuda(gpu)

        if abs(label_yaw_cont) > 99 or abs(label_pitch_cont) > 99 or abs(label_roll_cont) > 99:
            #print('out of range')
            continue
      
        x, yaw, pitch, roll = model(img)

        yaw_predicted = softmax(yaw)#,dim = 1)
        pitch_predicted = softmax(pitch)#,dim = 1)
        roll_predicted = softmax(roll)#,dim =1)

        yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 99
        pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 99
        roll_predicted = torch.sum(roll_predicted * idx_tensor, 1) * 3 - 99

        yaw_predicted = yaw_predicted.view(1)
        pitch_predicted = pitch_predicted.view(1)
        roll_predicted = roll_predicted.view(1)

        err_y = MAE(yaw_predicted,label_yaw_cont)
        err_p = MAE(pitch_predicted,label_pitch_cont)
        err_r = MAE(roll_predicted,label_roll_cont)

        MAE_LATENT_y = np.append(MAE_LATENT_y,err_y.cpu())
        MAE_LATENT_p = np.append(MAE_LATENT_p,err_p.cpu())
        MAE_LATENT_r = np.append(MAE_LATENT_r,err_r.cpu())

        if (i%1000 == 0):
            print(i)


In [13]:
print('MAE ERROR LATENT OCCLUSION AFWL2000:')
print('Yaw: ',sum(MAE_LATENT_y)/len(MAE_LATENT_y), ' Pitch: ',sum(MAE_LATENT_p)/len(MAE_LATENT_y), ' Roll: ',sum(MAE_LATENT_r)/len(MAE_LATENT_y))

MAE ERROR LATENT OCCLUSION AFWL2000:
Yaw:  4.644780082663132  Pitch:  6.165434748736537  Roll:  4.746627412617221


# BIWI OCCLUDED DATASET

In [7]:
image_path = natsort.natsorted(glob.glob('datasets\faces_biwi_wide_oclusion\*.jpg'),reverse= False)


pose_path = natsort.natsorted(glob.glob('datasets\faces_biwi\01\*.txt'),reverse= False) + natsort.natsorted(glob.glob('datasets\faces_biwi\02\*.txt'),reverse= False) + natsort.natsorted(glob.glob('datasets\faces_biwi\03\*.txt'),reverse= False) + natsort.natsorted(glob.glob('datasets\faces_biwi\04\*.txt'),reverse= False)+ natsort.natsorted(glob.glob('datasets\faces_biwi\05\*.txt'),reverse= False)


transformations = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])



biwi_test_dataset = datasets.new_BIWI(image_path,pose_path, transformations)


test_loader = torch.utils.data.DataLoader(dataset=biwi_test_dataset,
                                               batch_size=1,
                                               shuffle=False,
                                               num_workers=0)
print(len(image_path))
print('done')

# BIWI DATASET TEST

In [8]:
MAE_LATENT_y = np.array([])
MAE_LATENT_p = np.array([])
MAE_LATENT_r = np.array([])


num_epochs = 1
with torch.no_grad():
    for i, (img, labels, cont_labels) in enumerate(test_loader):

        img = Variable(img).cuda(gpu)

        label_yaw_cont = Variable(cont_labels[:,0]).cuda(gpu)
        label_pitch_cont = Variable(cont_labels[:,1]).cuda(gpu)
        label_roll_cont = Variable(cont_labels[:,2]).cuda(gpu)

        if abs(label_yaw_cont) > 99 or abs(label_pitch_cont) > 99 or abs(label_roll_cont) > 99:
            #print('out of range')
            continue
      
        x, yaw, pitch, roll = model(img)

        yaw_predicted = softmax(yaw)#,dim = 1)
        pitch_predicted = softmax(pitch)#,dim = 1)
        roll_predicted = softmax(roll)#,dim =1)

        yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 99
        pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 99
        roll_predicted = torch.sum(roll_predicted * idx_tensor, 1) * 3 - 99

        yaw_predicted = yaw_predicted.view(1)
        pitch_predicted = pitch_predicted.view(1)
        roll_predicted = roll_predicted.view(1)

        err_y = MAE(yaw_predicted,label_yaw_cont)
        err_p = MAE(pitch_predicted,label_pitch_cont)
        err_r = MAE(roll_predicted,label_roll_cont)

        MAE_LATENT_y = np.append(MAE_LATENT_y,err_y.cpu())
        MAE_LATENT_p = np.append(MAE_LATENT_p,err_p.cpu())
        MAE_LATENT_r = np.append(MAE_LATENT_r,err_r.cpu())

        if (i%1000 == 0):
            print(i)

In [9]:
print('MAE ERROR LATENT OCCLUSION BIWI:')
print('Yaw: ',sum(MAE_LATENT_y)/len(MAE_LATENT_y), ' Pitch: ',sum(MAE_LATENT_p)/len(MAE_LATENT_y), ' Roll: ',sum(MAE_LATENT_r)/len(MAE_LATENT_y))

MAE ERROR LATENT BIWI:
Yaw:  5.174480246811149  Pitch:  6.622005143735569  Roll:  4.116956870948608


# AFLW2000 NO OCCLUSION TEST

In [15]:
image_path = natsort.natsorted(glob.glob('datasets\AFLW2000_3D_Full_Faces\*.jpg'))

pose_path = natsort.natsorted(glob.glob('datasets\AFLW2000-3D\AFLW2000\*.mat'))

transformations =  transforms.Compose([transforms.Resize(260), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

test_dataset = datasets.new_Pose_AFLW2000(image_path, pose_path, transformations)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=1,shuffle=False, num_workers=0)
print(len(image_path))
print('done')

2000
done


In [16]:
MAE_LATENT_y = np.array([])
MAE_LATENT_p = np.array([])
MAE_LATENT_r = np.array([])


num_epochs = 1
with torch.no_grad():
    for i, (img, labels, cont_labels) in enumerate(test_loader):

        img = Variable(img).cuda(gpu)

        label_yaw_cont = Variable(cont_labels[:,0]).cuda(gpu)
        label_pitch_cont = Variable(cont_labels[:,1]).cuda(gpu)
        label_roll_cont = Variable(cont_labels[:,2]).cuda(gpu)

        if abs(label_yaw_cont) > 99 or abs(label_pitch_cont) > 99 or abs(label_roll_cont) > 99:
            #print('out of range')
            continue
      
        x, yaw, pitch, roll = model(img)

        yaw_predicted = softmax(yaw)#,dim = 1)
        pitch_predicted = softmax(pitch)#,dim = 1)
        roll_predicted = softmax(roll)#,dim =1)

        yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 99
        pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 99
        roll_predicted = torch.sum(roll_predicted * idx_tensor, 1) * 3 - 99

        yaw_predicted = yaw_predicted.view(1)
        pitch_predicted = pitch_predicted.view(1)
        roll_predicted = roll_predicted.view(1)

        err_y = MAE(yaw_predicted,label_yaw_cont)
        err_p = MAE(pitch_predicted,label_pitch_cont)
        err_r = MAE(roll_predicted,label_roll_cont)

        MAE_LATENT_y = np.append(MAE_LATENT_y,err_y.cpu())
        MAE_LATENT_p = np.append(MAE_LATENT_p,err_p.cpu())
        MAE_LATENT_r = np.append(MAE_LATENT_r,err_r.cpu())

        if (i%1000 == 0):
            print(i)

In [17]:
print('MAE ERROR LATENT NO OCCLUSION AFLW2000:')
print('Yaw: ',sum(MAE_LATENT_y)/len(MAE_LATENT_y), ' Pitch: ',sum(MAE_LATENT_p)/len(MAE_LATENT_y), ' Roll: ',sum(MAE_LATENT_r)/len(MAE_LATENT_y))

MAE ERROR LATENT NO OCCLUSION AFLW2000:
Yaw:  3.7374907772687256  Pitch:  5.415608577725395  Roll:  3.9669136630022144


# BIWI NO OCCLUSION TEST

In [18]:
image_path = natsort.natsorted(glob.glob('datasets\faces_biwi_wide\*.jpg'),reverse= False)

pose_path = natsort.natsorted(glob.glob('datasets\faces_biwi\01\*.txt'),reverse= False) + natsort.natsorted(glob.glob('datasets\faces_biwi\02\*.txt'),reverse= False) + natsort.natsorted(glob.glob('datasets\faces_biwi\03\*.txt'),reverse= False) + natsort.natsorted(glob.glob('datasets\faces_biwi\04\*.txt'),reverse= False)+ natsort.natsorted(glob.glob('datasets\faces_biwi\05\*.txt'),reverse= False)


transformations = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


biwi_test_dataset = datasets.new_BIWI(image_path,pose_path, transformations)


test_loader = torch.utils.data.DataLoader(dataset=biwi_test_dataset,
                                               batch_size=1,
                                               shuffle=False,
                                               num_workers=0)
print(len(image_path))
print('done')

3430
done


In [19]:
MAE_LATENT_y = np.array([])
MAE_LATENT_p = np.array([])
MAE_LATENT_r = np.array([])


num_epochs = 1
with torch.no_grad():
    for i, (img, labels, cont_labels) in enumerate(test_loader):

        img = Variable(img).cuda(gpu)

        label_yaw_cont = Variable(cont_labels[:,0]).cuda(gpu)
        label_pitch_cont = Variable(cont_labels[:,1]).cuda(gpu)
        label_roll_cont = Variable(cont_labels[:,2]).cuda(gpu)

        if abs(label_yaw_cont) > 99 or abs(label_pitch_cont) > 99 or abs(label_roll_cont) > 99:
            #print('out of range')
            continue
      
        x, yaw, pitch, roll = model(img)

        yaw_predicted = softmax(yaw)#,dim = 1)
        pitch_predicted = softmax(pitch)#,dim = 1)
        roll_predicted = softmax(roll)#,dim =1)

        yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 99
        pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 99
        roll_predicted = torch.sum(roll_predicted * idx_tensor, 1) * 3 - 99

        yaw_predicted = yaw_predicted.view(1)
        pitch_predicted = pitch_predicted.view(1)
        roll_predicted = roll_predicted.view(1)

        err_y = MAE(yaw_predicted,label_yaw_cont)
        err_p = MAE(pitch_predicted,label_pitch_cont)
        err_r = MAE(roll_predicted,label_roll_cont)

        MAE_LATENT_y = np.append(MAE_LATENT_y,err_y.cpu())
        MAE_LATENT_p = np.append(MAE_LATENT_p,err_p.cpu())
        MAE_LATENT_r = np.append(MAE_LATENT_r,err_r.cpu())

        if (i%1000 == 0):
            print(i)

C:\Users\zecar\AppData\Local\Temp/ipykernel_31492/24992921.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  yaw_predicted = softmax(yaw)#,dim = 1)
C:\Users\zecar\AppData\Local\Temp/ipykernel_31492/24992921.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pitch_predicted = softmax(pitch)#,dim = 1)
C:\Users\zecar\AppData\Local\Temp/ipykernel_31492/24992921.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  roll_predicted = softmax(roll)#,dim =1)


0
1000
2000
3000


In [20]:
print('MAE ERROR LATENT NO OCCLUSION BIWI:')
print('Yaw: ',sum(MAE_LATENT_y)/len(MAE_LATENT_y), ' Pitch: ',sum(MAE_LATENT_p)/len(MAE_LATENT_y), ' Roll: ',sum(MAE_LATENT_r)/len(MAE_LATENT_y))

MAE ERROR LATENT NO OCCLUSION BIWI:
Yaw:  4.296622778452477  Pitch:  4.586208105234989  Roll:  3.617039893832568
